# Geração das respostas e métricas com o Gemini

In [173]:
import pandas as pd

df = pd.read_csv(
    '../../data/inference_data.csv',
    encoding='utf-8',
    sep=','
)

# Converter register_time para datetime se não estiver
df['register_time'] = pd.to_datetime(df['register_time'])

# Verificar se o dataset está ordenado por register_time
print("=== VERIFICANDO ORDENAÇÃO DO DATASET ===")
print(f"Total de registros: {len(df)}")
print(f"Primeiros 5 registros (register_time):")
print(df[['register_time', 'title']].head(10))

# Verificar se está ordenado por register_time
is_sorted = df['register_time'].is_monotonic_increasing

# Se não estiver ordenado, vamos identificar onde estão os problemas
if not is_sorted:
    print("\n=== PROBLEMAS DE ORDENAÇÃO DETECTADOS ===")

    # Encontrar onde a ordem está quebrada
    problems = []
    for i in range(1, len(df)):
        if df.iloc[i]['register_time'] < df.iloc[i-1]['register_time']:
            problems.append({
                'index': i,
                'current_time': df.iloc[i]['register_time'],
                'previous_time': df.iloc[i-1]['register_time'],
                'current_task': df.iloc[i]['title'],
                'previous_task': df.iloc[i-1]['title']
            })

    print(f"Encontrados {len(problems)} problemas de ordenação:")
    for problem in problems:  # Mostrar apenas os primeiros 10
        print(f"  Linha {problem['index']}: {problem['current_time']} < {problem['previous_time']}")
        print(f"    Tarefa atual: {problem['current_task']}")
        print(f"    Tarefa anterior: {problem['previous_task']}")
        print()

print(f"\nDataset está ordenado por register_time? {is_sorted}")

if not is_sorted:
    print("⚠️ Dataset não está ordenado. Ordenando por register_time...")
    df = df.sort_values(['register_time', 'start_time']).reset_index(drop=True)
    print("✅ Dataset ordenado com sucesso!")
else:
    print("✅ Dataset já está ordenado!")

print(f"\nPrimeiros 5 registros após verificação/ordenação:")
print(df[['register_time', 'start_time', 'title']].head(20))

df.info()

=== VERIFICANDO ORDENAÇÃO DO DATASET ===
Total de registros: 143
Primeiros 5 registros (register_time):
        register_time                             title
0 2017-11-27 06:58:00      Attend the school graduation
1 2018-08-07 11:25:00             Go to Baile da Gaiola
2 2020-09-18 00:58:00  Study calculus and problem set 2
3 2021-05-18 21:23:00    Go to psychiatrist appointment
4 2021-05-21 02:39:00          Facetime with Alessandra
5 2021-07-07 23:53:00                       Take exam 3
6 2021-09-01 15:56:00    Go to psychologist appointment
7 2021-10-25 11:01:00            Consult with Dr. Bruno
8 2022-01-17 13:36:00                     Go to the gym
9 2022-01-20 06:38:00                 Have chiropractic

=== PROBLEMAS DE ORDENAÇÃO DETECTADOS ===
Encontrados 11 problemas de ordenação:
  Linha 30: 2022-04-09 18:52:00 < 2022-04-14 10:16:00
    Tarefa atual: Have an imaging exam
    Tarefa anterior: Consult the dermatologist

  Linha 34: 2022-05-16 10:30:00 < 2022-05-16 10:39:00
   

In [178]:
# Converter blocos de 30 minutos (336 blocos) para blocos de 1 hora (168 blocos)
print("=== CONVERTENDO BLOCOS DE 30 MIN PARA 1 HORA ===")

# Verificar valores originais da coluna start_time_slot
print(f"Valores originais - Min: {df['start_time_slot'].min()}, Max: {df['start_time_slot'].max()}")
print(f"Total de valores únicos originais: {df['start_time_slot'].nunique()}")

# Fórmula de conversão:
# - Blocos de 30 min: 1-336 (segunda 00:00 até domingo 00:00)
# - Blocos de 1 hora: 1-168 (segunda 00:00 até domingo 00:00)
# - Conversão: bloco_1h = ceil(bloco_30min / 2)

import math

# Aplicar a conversão
df['start_time_slot_1h'] = df['start_time_slot'].apply(lambda x: math.ceil(x / 2))

# Verificar resultados da conversão
print(f"\nValores convertidos - Min: {df['start_time_slot_1h'].min()}, Max: {df['start_time_slot_1h'].max()}")
print(f"Total de valores únicos convertidos: {df['start_time_slot_1h'].nunique()}")

# Substituir a coluna original pela convertida
df['start_time_slot'] = df['start_time_slot_1h']
df = df.drop('start_time_slot_1h', axis=1)

print(f"\n✅ Conversão concluída!")
print(f"Coluna start_time_slot agora contém blocos de 1 hora (1-168)")

# Mostrar alguns exemplos da conversão
print(f"\nExemplos dos primeiros valores convertidos:")
print(df[['title', 'start_time_slot']].head(10))

=== CONVERTENDO BLOCOS DE 30 MIN PARA 1 HORA ===
Valores originais - Min: 6, Max: 326
Total de valores únicos originais: 92

Valores convertidos - Min: 3, Max: 163
Total de valores únicos convertidos: 75

✅ Conversão concluída!
Coluna start_time_slot agora contém blocos de 1 hora (1-168)

Exemplos dos primeiros valores convertidos:
                              title  start_time_slot
0      Attend the school graduation               95
1             Go to Baile da Gaiola              118
2  Study calculus and problem set 2              104
3    Go to psychiatrist appointment               67
4          Facetime with Alessandra              112
5                       Take exam 3               83
6    Go to psychologist appointment              112
7            Consult with Dr. Bruno               20
8                     Go to the gym               18
9                 Have chiropractic               83


In [174]:
def get_context_tasks(df: pd.DataFrame, current_index: int) -> list:
    """
    Para uma tarefa específica (current_index), retorna TODAS as tarefas anteriores como contexto.

    Args:
        df (pd.DataFrame): DataFrame com as tarefas ordenadas por register_time
        current_index (int): Índice da tarefa atual

    Returns:
        list: Lista de dicionários com TODAS as tarefas anteriores
    """
    if current_index == 0:
        return []  # Primeira tarefa não tem contexto

    # Pegar TODAS as tarefas anteriores à atual
    previous_tasks = df.iloc[:current_index]

    # Converter para lista de dicionários com informações relevantes
    context_tasks = []
    for _, task in previous_tasks.iterrows():
        context_task = {
            'email_address': task['email_address'],
            'title': task['title'],
            'duration_minute': task['duration_minute'],
            'register_time': task['register_time'],
            'start_iso_year': task['start_iso_year'],
            'start_iso_week': task['start_iso_week'],
            'week_register_sequence': task['week_register_sequence'],
            'start_time_slot': task['start_time_slot']
        }
        context_tasks.append(context_task)

    return context_tasks

# Exemplo de uso da função
print("=== TESTANDO A FUNÇÃO ===")

# Testar com a 10ª tarefa
example_index = 9
context_tasks = get_context_tasks(df, example_index)

current_task = df.iloc[example_index]
print(f"\n=== EXEMPLO - Tarefa {example_index + 1} ===")
print(f"Tarefa atual: {current_task['title']}")
print(f"Data de criação: {current_task['register_time']}")
print(f"Número de tarefas anteriores no contexto: {len(context_tasks)}")

if len(context_tasks) > 0:
    print(f"\nPrimeiras 3 tarefas do contexto:")
    for i, task in enumerate(context_tasks[:3]):
        print(f"  {i+1}. {task['title']} (criada em: {task['register_time']})")
        print(f"     Duração: {task['duration_minute']} min | Slot: {task['start_time_slot']}")

    if len(context_tasks) > 3:
        print(f"  ... e mais {len(context_tasks) - 3} tarefas no contexto")

print(f"\n=== EXEMPLO DE CONTEXTO COMPLETO DA PRIMEIRA TAREFA ===")
if len(context_tasks) > 0:
    first_task = context_tasks[0]
    print("Todas as informações da primeira tarefa do contexto:")
    for key, value in first_task.items():
        print(f"  {key}: {value}")

=== TESTANDO A FUNÇÃO ===

=== EXEMPLO - Tarefa 10 ===
Tarefa atual: Have chiropractic
Data de criação: 2022-01-20 06:38:00
Número de tarefas anteriores no contexto: 9

Primeiras 3 tarefas do contexto:
  1. Attend the school graduation (criada em: 2017-11-27 06:58:00)
     Duração: 480 min | Slot: 190
  2. Go to Baile da Gaiola (criada em: 2018-08-07 11:25:00)
     Duração: 480 min | Slot: 236
  3. Study calculus and problem set 2 (criada em: 2020-09-18 00:58:00)
     Duração: 300 min | Slot: 208
  ... e mais 6 tarefas no contexto

=== EXEMPLO DE CONTEXTO COMPLETO DA PRIMEIRA TAREFA ===
Todas as informações da primeira tarefa do contexto:
  email_address: sample@email.com
  title: Attend the school graduation
  duration_minute: 480
  register_time: 2017-11-27 06:58:00
  start_iso_year: 2017
  start_iso_week: 50
  week_register_sequence: 0
  start_time_slot: 190


In [175]:
import yaml

yaml_path = '../../scheduler.prompt.yml'
with open(yaml_path, 'r', encoding='utf-8') as file:
    verbalizer_config = yaml.safe_load(file)

# Acessar partes específicas do YAML
messages = verbalizer_config.get('messages', [])
model = verbalizer_config.get('model', '')
model_params = verbalizer_config.get('modelParameters', {})
temperature = verbalizer_config.get('modelParameters').get('temperature')
max_tokens = verbalizer_config.get('modelParameters').get('max_tokens')


print(f"Modelo: {model}")
print(f"Parâmetros do modelo: {model_params}")
print(f'Temperatura: {temperature}')
print(f'Máximo de tokens: {max_tokens}')

# Exemplo de como acessar o conteúdo das mensagens
print('Mensagens:')
print(f'System: {messages[0].get("content", "")}')
print(f'User: {messages[1].get("content", "")}')

Modelo: openai/gpt-4o
Parâmetros do modelo: {'max_tokens': 2048, 'temperature': 0.0, 'top_p': 0.8}
Temperatura: 0.0
Máximo de tokens: 2048
Mensagens:
System: Pense passo a passo como um especialista em agendamento de tarefas. Você receberá 12 colunas de atributos de uma tarefa a ser agendada, além do contexto de tarefas anteriores. Sua missão é escolher o bloco de horário mais adequado para essa tarefa, considerando as colunas e o contexto.
1. **Colunas**: - Coluna 1 **email_address**: Email do usuário que está agendando a tarefa - Coluna 2 **title**: Descrição da tarefa - Coluna 3 **duration_minute**: Duração estimada da tarefa (em minutos) - Coluna 4 **register_time**: Data de registro/criação da tarefa - Coluna 5 **start_time**: Data de início preferencial da tarefa - Coluna 6 **start_iso_year**: Ano ISO da data de início preferencial - Coluna 7 **start_iso_week**: Semana ISO da data de início preferencial - Coluna 8 **week_register_sequence**: Posição da tarefa na sequência de regi

In [176]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

In [177]:
import numpy as np

def schedule_task(task: dict, context_tasks: list) -> str:
    """
    Agendar uma tarefa com base no contexto de tarefas anteriores.

    Args:
        task (dict): Dicionário com os detalhes da tarefa a ser agendada.
        context_tasks (list): Lista de dicionários com as tarefas anteriores como contexto.

    Returns:
        str: Resposta do modelo sobre o agendamento da tarefa.
    """

    print('Tarefa: ', str(task))
    print('Contexto de tarefas: ', str(context_tasks))
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=[
            {
                "role": "system",
                "content": messages[0]['content']
                .replace('{{PREVIOUS_TASKS}}', str(context_tasks))
            },
            {
                "role": "user",
                "content": messages[1]['content']
                .replace('{{TASK}}', str(task))
            },
        ],
        temperature=temperature,
        logprobs=True,
        top_logprobs=5,
        # seed=42,
    )

    return response


# Exemplo de uso da função schedule_task
task = df.iloc[9].to_dict()  # Tarefa atual
filtered_task = {
    'email_address': task['email_address'],
    'title': task['title'],
    'duration_minute': task['duration_minute'],
}
context_tasks = get_context_tasks(df, 9)  # Contexto de tarefas anteriores
response = schedule_task(filtered_task, context_tasks)
print('Bloco de tempo original: ', task['start_time_slot'])

# ! Lembrar de sempre substrair 1 do bloco retornado pelo modelo
for logprob in response.choices[0].logprobs.content[0].top_logprobs:
    print(f'Bloco: {logprob.token} - LogProb: {logprob.logprob} - LinearProb: {np.round(np.exp(logprob.logprob)*100,2)}')


Tarefa:  {'email_address': 'sample@email.com', 'title': 'Have chiropractic', 'duration_minute': 120}
Contexto de tarefas:  [{'email_address': 'sample@email.com', 'title': 'Attend the school graduation', 'duration_minute': 480, 'register_time': Timestamp('2017-11-27 06:58:00'), 'start_iso_year': 2017, 'start_iso_week': 50, 'week_register_sequence': 0, 'start_time_slot': 190}, {'email_address': 'sample@email.com', 'title': 'Go to Baile da Gaiola', 'duration_minute': 480, 'register_time': Timestamp('2018-08-07 11:25:00'), 'start_iso_year': 2018, 'start_iso_week': 32, 'week_register_sequence': 10, 'start_time_slot': 236}, {'email_address': 'sample@email.com', 'title': 'Study calculus and problem set 2', 'duration_minute': 300, 'register_time': Timestamp('2020-09-18 00:58:00'), 'start_iso_year': 2020, 'start_iso_week': 38, 'week_register_sequence': 0, 'start_time_slot': 208}, {'email_address': 'sample@email.com', 'title': 'Go to psychiatrist appointment', 'duration_minute': 90, 'register_ti

# Função para obter contexto de tarefas

## Função criada:

**`get_context_tasks(df, current_index)`**: 
- Pega TODAS as tarefas anteriores para uma tarefa específica
- Dataset deve estar ordenado por `register_time` (data de criação)
- Retorna lista de dicionários com TODAS as tarefas de contexto
- Sem limitação de quantidade - inclui todas as tarefas anteriores

## Como usar:

```python
# Para cada linha do dataframe
for index in range(len(df)):
    current_task = df.iloc[index]
    context_tasks = get_context_tasks(df, index)
    
    # Usar context_tasks na sua função de agendamento
    # context_tasks contém TODAS as tarefas anteriores
```

## Informações completas no contexto:

Cada tarefa no contexto contém TODAS as colunas disponíveis:
- `email_address`: endereço de email do usuário
- `title`: descrição da tarefa
- `duration_minute`: duração em minutos  
- `register_time`: quando foi registrada
- `start_time`: quando foi iniciada
- `start_iso_year`: ano ISO da data de início
- `start_iso_week`: semana ISO da data de início
- `week_register_sequence`: sequência de registro na semana
- `register_start_week_distance`: distância em semanas entre registro e início
- `register_start_day_distance`: distância em dias entre registro e início
- `is_recurrent`: se é recorrente
- `start_time_slot`: slot de tempo escolhido